In [76]:
import re
import nltk
nltk.download('omw-1.4')
from gensim.test.utils import common_texts
# import data as data
from gensim.models import Word2Vec
import pandas as pd
import pandas as pd
pd.set_option('display.max_rows', 2500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\avitr\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [77]:
df = pd.read_csv("autotagger.tsv",sep='\t')
df.head(30)

,product_id,product_name,kw_name,description
0,126879,Addis Flip Top 46L Black Bin,NaN,Dispose of unwanted rubbish without having to ...
1,127376,Arnott's Tim Tam Chewy Caramel 200g,NaN,NaN
2,128411,Beeno Flatties Chicken 120g,NaN,NaN
3,138386,Beyers Amarula Chocolate 110g,NaN,NaN
4,138490,Bob Martin 2 In 1 Shampoo And Conditioner 400ml,NaN,NaN
5,136934,Bob Martin Crystal Cat Litter Lavender 1.8kg,NaN,NaN
6,140919,BRM PERI-PERI SPATCHCOCK CHICKEN 1.2KG,NaN,Fully cooked;\nReady in minutes;\nCooked until...
7,122149,BRM Pork Belly Ribs Bbq 2kg,ToBeRemoved,Pre-cooked marinated in bbq sauce
8,122931,Buffalo Citric Acid 100g,NaN,NaN
9,135665,Carmien Organic Teabags Pouch 100ea,NaN,NaN


In [18]:
df.columns

Index(['product_id', 'product_name', 'kw_name', 'description'], dtype='object')

In [21]:
sum(list(df['description'].value_counts()))

1868

In [24]:
df.isna().sum()

product_id          0
product_name        0
kw_name          8191
description     11767
dtype: int64

In [16]:
df_X = df[['product_name','description']].dropna()
df_y = df[df['description'].isna()]

In [17]:
df_X

,product_name,description
0,Addis Flip Top 46L Black Bin,Dispose of unwanted rubbish without having to ...
6,BRM PERI-PERI SPATCHCOCK CHICKEN 1.2KG,Fully cooked;\nReady in minutes;\nCooked until...
7,BRM Pork Belly Ribs Bbq 2kg,Pre-cooked marinated in bbq sauce
11,Crushed Ginger 130g,Perfect for stews & curries!
12,D'Usse VSOP Cognac 750ml,D'Usse VSOP Cognac 750ml
...,...,...
13570,Ybarra Extra Virgin Olive Oil 1L,Olive Oil
13597,Yum Yum Smooth Peanut Butter 400g,Give your sandwich some flavour with this amaz...
13598,Yum Yum Smooth Peanut Butter 400g,Give your sandwich some flavour with this amaz...
13599,Yum Yum Smooth Peanut Butter 400g,Give your sandwich some flavour with this amaz...


In [20]:
df['kw_name'].value_counts().index

Index(['ToBeRemoved', 'Wine', 'chicken', 'wines', 'Red Wine', 'meat', 'drink',
       'liquor', 'snacks', 'Milk',
       ...
       'sparkling water', 'cleaning', 'delicious', 'coffe', 'flour', 'pumo',
       'bar', 'tant sannie', 'mac & cheese', 'kids'],
      dtype='object', length=376)

In [22]:
df['product_id'].value_counts()

64648     8
64465     7
64291     6
64955     6
67837     6
         ..
95812     1
65093     1
140870    1
67146     1
67583     1
Name: product_id, Length: 11388, dtype: int64

In [23]:
df[df['product_id']==64648]

,product_id,product_name,kw_name,description
11124,64648,Sanex Dermo Kids Biome Protect Body Wash & Foa...,ToBeRemoved,Sanex Dermo Kids Biome Protect Body Wash & Foa...
11125,64648,Sanex Dermo Kids Biome Protect Body Wash & Foa...,shower gel,Sanex Dermo Kids Biome Protect Body Wash & Foa...
11126,64648,Sanex Dermo Kids Biome Protect Body Wash & Foa...,soap,Sanex Dermo Kids Biome Protect Body Wash & Foa...
11127,64648,Sanex Dermo Kids Biome Protect Body Wash & Foa...,bathing,Sanex Dermo Kids Biome Protect Body Wash & Foa...
11128,64648,Sanex Dermo Kids Biome Protect Body Wash & Foa...,kids bath,Sanex Dermo Kids Biome Protect Body Wash & Foa...
11129,64648,Sanex Dermo Kids Biome Protect Body Wash & Foa...,kid,Sanex Dermo Kids Biome Protect Body Wash & Foa...
11130,64648,Sanex Dermo Kids Biome Protect Body Wash & Foa...,kids,Sanex Dermo Kids Biome Protect Body Wash & Foa...
11131,64648,Sanex Dermo Kids Biome Protect Body Wash & Foa...,kids batch,Sanex Dermo Kids Biome Protect Body Wash & Foa...


In [29]:
df.fillna('anything')

,product_id,product_name,kw_name,description
0,126879,Addis Flip Top 46L Black Bin,anything,Dispose of unwanted rubbish without having to ...
1,127376,Arnott's Tim Tam Chewy Caramel 200g,anything,anything
2,128411,Beeno Flatties Chicken 120g,anything,anything
3,138386,Beyers Amarula Chocolate 110g,anything,anything
4,138490,Bob Martin 2 In 1 Shampoo And Conditioner 400ml,anything,anything
...,...,...,...,...
13630,93196,Zonnebloem Pinotage 750ml,Red Wine,anything
13631,93196,Zonnebloem Pinotage 750ml,Wine,anything
13632,93196,Zonnebloem Pinotage 750ml,wines,anything
13633,115640,Zooty Groovy 60ml,anything,anything


In [31]:
df.isna().sum()

product_id          0
product_name        0
kw_name          8191
description     11767
dtype: int64

In [86]:
t = df[df['description'].isna() == False]

In [87]:
t.isna().sum()

product_id        0
product_name      0
kw_name         950
description       0
dtype: int64

In [88]:
len(t) - 950

918

In [89]:
t

,product_id,product_name,kw_name,description
0,126879,Addis Flip Top 46L Black Bin,NaN,Dispose of unwanted rubbish without having to ...
6,140919,BRM PERI-PERI SPATCHCOCK CHICKEN 1.2KG,NaN,Fully cooked;\nReady in minutes;\nCooked until...
7,122149,BRM Pork Belly Ribs Bbq 2kg,ToBeRemoved,Pre-cooked marinated in bbq sauce
11,140932,Crushed Ginger 130g,NaN,Perfect for stews & curries!
12,141080,D'Usse VSOP Cognac 750ml,NaN,D'Usse VSOP Cognac 750ml
13,141432,Dettol Liquid Bathroom Cleaner Ocean Fresh 50...,NaN,\nDettol Liquid Bathroom Cleaner Ocean Fresh 5...
17,141000,Frozen Mixed Berries 1Kg,NaN,specially and seleceted\n
18,140252,Glad Small Zipper Freezer Bags 15,NaN,"Glad Freezer Bags are thick, strong and suitab..."
21,141487,Johnnie Walker Black Label 750ml,NaN,Whisky
31,140985,Oranges 4pk,NaN,specially and selected


In [90]:
t = t[t['kw_name']!='ToBeRemoved']

In [91]:
data = t.copy()

In [148]:
data.shape

(1525, 4)

In [93]:
data.reset_index(drop=True,inplace=True)

In [94]:
data

,product_id,product_name,kw_name,description
0,126879,Addis Flip Top 46L Black Bin,NaN,Dispose of unwanted rubbish without having to ...
1,140919,BRM PERI-PERI SPATCHCOCK CHICKEN 1.2KG,NaN,Fully cooked;\nReady in minutes;\nCooked until...
2,140932,Crushed Ginger 130g,NaN,Perfect for stews & curries!
3,141080,D'Usse VSOP Cognac 750ml,NaN,D'Usse VSOP Cognac 750ml
4,141432,Dettol Liquid Bathroom Cleaner Ocean Fresh 50...,NaN,\nDettol Liquid Bathroom Cleaner Ocean Fresh 5...
5,141000,Frozen Mixed Berries 1Kg,NaN,specially and seleceted\n
6,140252,Glad Small Zipper Freezer Bags 15,NaN,"Glad Freezer Bags are thick, strong and suitab..."
7,141487,Johnnie Walker Black Label 750ml,NaN,Whisky
8,140985,Oranges 4pk,NaN,specially and selected
9,140976,Qualifresh Crushed Garlic 125g,NaN,Specially and selected


In [95]:
df_train = data[data['kw_name'].isna() == False]
df_test = data[data['kw_name'].isna() == True]

In [97]:
df_train.isna().sum()

product_id      0
product_name    0
kw_name         0
description     0
dtype: int64

In [101]:
df_train.shape

(575, 4)

In [99]:
df_test.shape

(950, 4)

In [98]:
df_test.isna().sum()

product_id        0
product_name      0
kw_name         950
description       0
dtype: int64

# Model

In [103]:
df_train.reset_index(drop=True,inplace=True)

In [104]:
df_test.reset_index(drop=True,inplace=True)

In [109]:
x = df_train['description'][0]

In [115]:
from gensim.parsing.preprocessing import preprocess_string

In [124]:
#https://radimrehurek.com/gensim/parsing/preprocessing.html
all_description = []
for description in df_train['description'].values:
    all_description.append(preprocess_string(description))

In [125]:
all_description

[['safe',
  'heaalthi',
  'clean',
  'hand',
  'power',
  'hygien',
  'waterless',
  'hand',
  'gel'],
 ['safe',
  'heaalthi',
  'clean',
  'hand',
  'power',
  'hygien',
  'waterless',
  'hand',
  'gel'],
 ['safe',
  'heaalthi',
  'clean',
  'hand',
  'power',
  'hygien',
  'waterless',
  'hand',
  'gel'],
 ['safe',
  'heaalthi',
  'clean',
  'hand',
  'power',
  'hygien',
  'waterless',
  'hand',
  'gel'],
 ['safe',
  'heaalthi',
  'clean',
  'hand',
  'power',
  'hygien',
  'waterless',
  'hand',
  'gel'],
 ['addi',
  'floor',
  'broom',
  'natur',
  'nfloor',
  'broom',
  'synthet',
  'bristl',
  'colour',
  'end',
  'nfeatur',
  'handl',
  'includ',
  'metal',
  'hanger',
  'easi',
  'storag',
  'synthet',
  'bristl',
  'colour',
  'end',
  'nspecif',
  'materi',
  'plastic',
  'colour',
  'natur',
  'dimens',
  'weight',
  'warranti',
  'base',
  'manufactur',
  'default'],
 ['addi',
  'floor',
  'broom',
  'natur',
  'nfloor',
  'broom',
  'synthet',
  'bristl',
  'colour',
  'e

In [ ]:
from gensim.models import Word2Vec

In [138]:
model = Word2Vec(all_description, 
                 min_count=1,   # word frequency
                 size=10,      # dimention of word embeddings
                 workers=8,     # Number of processors
                 sg=1,
                 window=2,      # Context window
                )      

In [139]:
# save
model.wv.save_word2vec_format('w2vec_10d.txt')

# How to load:
from gensim.models import KeyedVectors
w2v = KeyedVectors.load_word2vec_format('w2vec_10d.txt')

In [140]:
w2v['safe'].shape

(10,)

In [141]:
vector = []
for line in all_description:
    x = np.zeros(10)
    for word in line:
        x = x + w2v[word]
    vector.append(x/len(line))

In [144]:
df_train['vector'] = vector

<ipython-input-144-98b88a6a4c9a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['vector'] = vector


In [145]:
df_train

,product_id,product_name,kw_name,description,vector
0,140132,Adco Hygiene Waterless Hand Gel 500ml,sanitiser,Always keep yourself safe and heaalthy by clea...,"[0.019916037356273994, -0.04181378197649287, 0..."
1,140132,Adco Hygiene Waterless Hand Gel 500ml,sanitizer,Always keep yourself safe and heaalthy by clea...,"[0.019916037356273994, -0.04181378197649287, 0..."
2,140132,Adco Hygiene Waterless Hand Gel 500ml,disinfectant,Always keep yourself safe and heaalthy by clea...,"[0.019916037356273994, -0.04181378197649287, 0..."
3,140132,Adco Hygiene Waterless Hand Gel 500ml,hand wash,Always keep yourself safe and heaalthy by clea...,"[0.019916037356273994, -0.04181378197649287, 0..."
4,140132,Adco Hygiene Waterless Hand Gel 500ml,wash,Always keep yourself safe and heaalthy by clea...,"[0.019916037356273994, -0.04181378197649287, 0..."
5,64797,Addis Floor Broom,floor cleaning,Addis - Floor Broom - Natural\n\nFloor broom. ...,"[0.012710765411611646, -0.025087047441047616, ..."
6,64797,Addis Floor Broom,broom,Addis - Floor Broom - Natural\n\nFloor broom. ...,"[0.012710765411611646, -0.025087047441047616, ..."
7,64799,Addis Mop 300g,floor cleaning,Addis Anti Bacterial Mop - 300g\n\nWelcome to ...,"[0.02478900217209701, -0.02094866799328614, 0...."
8,64799,Addis Mop 300g,mops,Addis Anti Bacterial Mop - 300g\n\nWelcome to ...,"[0.02478900217209701, -0.02094866799328614, 0...."
9,122230,Albany Hotdog Brown Rolls 6s,hot dogs,"Our brown buns are made with love, and we bake...","[0.012522966268339328, -0.015627597026260837, ..."


In [146]:
df_test

,product_id,product_name,kw_name,description
0,126879,Addis Flip Top 46L Black Bin,NaN,Dispose of unwanted rubbish without having to ...
1,140919,BRM PERI-PERI SPATCHCOCK CHICKEN 1.2KG,NaN,Fully cooked;\nReady in minutes;\nCooked until...
2,140932,Crushed Ginger 130g,NaN,Perfect for stews & curries!
3,141080,D'Usse VSOP Cognac 750ml,NaN,D'Usse VSOP Cognac 750ml
4,141432,Dettol Liquid Bathroom Cleaner Ocean Fresh 50...,NaN,\nDettol Liquid Bathroom Cleaner Ocean Fresh 5...
5,141000,Frozen Mixed Berries 1Kg,NaN,specially and seleceted\n
6,140252,Glad Small Zipper Freezer Bags 15,NaN,"Glad Freezer Bags are thick, strong and suitab..."
7,141487,Johnnie Walker Black Label 750ml,NaN,Whisky
8,140985,Oranges 4pk,NaN,specially and selected
9,140976,Qualifresh Crushed Garlic 125g,NaN,Specially and selected


In [150]:
import os

In [151]:
os.cpu_count()

8